In [13]:

!pip install datasets transformers librosa soundfile numpy"<"2 torch torchaudio fast-tsp kmedoids


  Obtaining dependency information for kmedoids from https://files.pythonhosted.org/packages/6e/b4/457e552aa362cc746fbf0c461ab1a9bd7794db3321f82586601769f27112/kmedoids-0.5.3.1-cp312-cp312-macosx_10_12_x86_64.macosx_11_0_arm64.macosx_10_12_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 kB 9.3 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# Setup

In [84]:
%load_ext autoreload
%autoreload 2

from transformers import ClapModel, ClapFeatureExtractor, AutoTokenizer

device='mps'
model = ClapModel.from_pretrained("laion/clap-htsat-unfused", use_safetensors=True).to(device)
feature_extractor: ClapFeatureExtractor = ClapFeatureExtractor.from_pretrained("laion/clap-htsat-unfused")
tokenizer = AutoTokenizer.from_pretrained("laion/clap-htsat-unfused")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

# Run

In [198]:
import torchaudio
import torchaudio.transforms as T

filename = '/Users/damian/Downloads/Dua Lipa - Be The One (Official Music Video).mp3'
waveform, sampling_rate = torchaudio.load(filename)
chunk_beats = 2
bpm = 87.5
window_width = 0

if sampling_rate != feature_extractor.sampling_rate:
    resampler = T.Resample(sampling_rate,  feature_extractor.sampling_rate, dtype=waveform.dtype)
    waveform, sampling_rate = resampler(waveform), feature_extractor.sampling_rate

print(sampling_rate, waveform.shape)
#inputs = feature_extractor(waveform, sampling_rate=sample_rate, return_tensors="pt")
#audio_features = model.get_audio_features(**inputs)

48000 torch.Size([2, 9805880])


In [199]:
from typing import Generator
import torch
torch.set_grad_enabled(False)
from tqdm.auto import tqdm

def wrap(idx, total):
    if idx < 0:
        return total + idx
    elif idx >= total:
        return idx - total
    else:
        return idx
    
def get_audio_chunks(waveform, chunk_size_seconds: float=0.25, 
                     window_width: float=0, include_window: bool=True
             ) -> Generator[torch.Tensor, None, None]:
    if len(waveform.shape) != 1:
        raise ValueError("waveform should have shape [num_samples]")
    chunk_size = int(8 * ((sampling_rate * chunk_size_seconds) // 8))
    window_width_samples = int(chunk_size * window_width)
    for offset in range(window_width_samples, waveform.shape[0]-window_width_samples, chunk_size):
        if include_window:
            start = wrap(offset-window_width_samples, waveform.shape[0])
            end = wrap(offset+chunk_size+window_width_samples, waveform.shape[0])
        else:
            start = offset
            end = offset + chunk_size
        yield waveform[start:end]
    

def get_audio_features(waveform: torch.Tensor, sampling_rate):
    inputs = feature_extractor(waveform, sampling_rate=sampling_rate, return_tensors="pt")
    #print(inputs.keys())
    audio_features = model.get_audio_features(input_features=inputs.input_features.to(device))
    return audio_features / torch.norm(audio_features, p=2, dim=1, keepdim=True)

def get_text_features(text: str):
    inputs = tokenizer(text, padding=True, return_tensors='pt')
    text_features =  model.get_text_features(input_ids = inputs.input_ids.to(device))
    return text_features / torch.norm(text_features, p=2, dim=1, keepdim=True)


In [200]:
import pickle
import os

chunk_size_seconds = chunk_beats*60/bpm
left_chunks_window = list(get_audio_chunks(waveform[0], chunk_size_seconds=chunk_size_seconds, window_width=window_width))[:-1]
right_chunks_window = list(get_audio_chunks(waveform[1], chunk_size_seconds=chunk_size_seconds, window_width=window_width))[:-1]
mono_chunks = [(left_chunks_window[i] + right_chunks_window[i]) / 2 for i in range(len(left_chunks_window))]

left_chunks_no_window = list(get_audio_chunks(waveform[0], chunk_size_seconds=chunk_size_seconds, window_width=window_width, include_window=False))[:-1]
right_chunks_no_window = list(get_audio_chunks(waveform[1], chunk_size_seconds=chunk_size_seconds, window_width=window_width, include_window=False))[:-1]
stereo_chunks_no_window = [torch.stack([left_chunks_no_window[index], right_chunks_no_window[index]])
    for index in range(len(left_chunks_no_window))]
features_pickle_filename = filename + f'.clap-norm-bpm{bpm}-cb{chunk_beats}-ww{window_width}.pkl'
if os.path.exists(features_pickle_filename):
    with open(features_pickle_filename, 'rb') as f:
        all_features = pickle.load(f)
else:
    all_features = torch.concat([get_audio_features(chunk, sampling_rate=sampling_rate) 
                             for chunk in tqdm(mono_chunks)])
    with open(features_pickle_filename, 'wb') as f:
        pickle.dump(all_features, f)



In [201]:
from src.clap_slice.medoids_tsp import sort_tsp
sort_order = sort_tsp(all_features)

computing distance matrix
computing route


In [202]:
chunks_sorted = [torch.stack([left_chunks_no_window[index], right_chunks_no_window[index]])
    for index in sort_order]
print(chunks_sorted[0].shape)
sorted_audio = torch.cat(chunks_sorted, dim=1)
print(sorted_audio.shape)

torch.Size([2, 65824])
torch.Size([2, 9741952])


In [ ]:
"""sort_order = torch.tensor([ 96,  40, 128, 123, 115, 127,  83,  27,  31,  87,  71,  67,  75, 100,
         44, 102,  46, 134, 132,  51, 139, 107, 111,  55, 143,  43, 131,  99,
         41,  97, 129, 101, 133,  45,  47, 103, 135, 136, 104,  48, 140, 108,
         52, 110,  54, 106,  50, 138, 105,  49, 137, 109, 141,  53, 142,  78,
         77,  69,  85,  29,  30,  86,  70,  59,  63,  65,  64,  72,  74,  66,
         76,  68,  73,  56,  60,  61,  57,  62,  58,  21,  22,  13,  14,  39,
         95,  38,  94,  93,  37, 144, 147, 146, 145,   0,   3,   4,   6,   5,
          2,   1,   7,  15,  16,  17,  20,  12,  10,   8,   9,  18, 122, 126,
         19,  11, 119, 116, 124, 120, 125, 121, 117, 118, 114, 113, 112,  92,
         36,  23,  79,  89,  81,  25,  33,  32,  88,  24,  80,  90,  34,  82,
         26,  28,  84,  35,  91, 130,  42,  98])"""
None

In [203]:
sort_order

tensor([ 96,  40, 128, 123, 115, 127,  83,  27,  31,  87,  71,  67,  75, 100,
         44, 102,  46, 134, 132,  51, 139, 107, 111,  55, 143,  43, 131,  99,
         41,  97, 129, 101, 133,  45,  47, 103, 135, 136, 104,  48, 140, 108,
         52, 110,  54, 106,  50, 138, 105,  49, 137, 109, 141,  53, 142,  78,
         77,  69,  85,  29,  30,  86,  70,  59,  63,  65,  64,  72,  74,  66,
         76,  68,  73,  56,  60,  61,  57,  62,  58,  21,  22,  13,  14,  39,
         95,  38,  94,  93,  37, 144, 147, 146, 145,   0,   3,   4,   6,   5,
          2,   1,   7,  15,  16,  17,  20,  12,  10,   8,   9,  18, 122, 126,
         19,  11, 119, 116, 124, 120, 125, 121, 117, 118, 114, 113, 112,  92,
         36,  23,  79,  89,  81,  25,  33,  32,  88,  24,  80,  90,  34,  82,
         26,  28,  84,  35,  91, 130,  42,  98])

In [204]:
torchaudio.save(filename + f'-sorted-norm-bpm{bpm}-cb{chunk_beats}-ww{window_width}.wav', sorted_audio, sample_rate=sampling_rate)

In [205]:
from typing import Literal


def smear(chunks, smear_width):
    envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
    #for i in range(smear_width+1, 2*smear_width+1):
    #    envelope[i] = 0
    print(envelope)
    for i in range(len(chunks)-smear_width*2-1):
        smear_sources = [
            chunks[i+chunk_index] * envelope[chunk_index]
            #(i+chunk_index, envelope[chunk_index])
            for chunk_index in range(0, min(len(chunks), smear_width*2+1))
        ]
        #print(smear_sources)
        #print(set([ss.shape for ss in smear_sources]))
        #print(torch.sum(torch.stack(smear_sources), dim=0))
        smeared_chunk = torch.sum(torch.stack(smear_sources), dim=0) / len(smear_sources)
        #print(smeared_chunk.shape)
        #break
        yield smeared_chunk
        
def smear_2(original_order_chunks_no_window, sort_order, smear_width, spread=4, 
            smear_mode=Literal['in', 'in-out'],
            dynamic_smear_width_embeds: tuple[torch.Tensor, list[int]]=None):
    
    def get_envelope(smear_width):
        envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
        if smear_mode == 'in':
            for i in range(smear_width+1, 2*smear_width+1):
                envelope[i] = 0
        return envelope
    envelope = get_envelope(smear_width)
    print(envelope)
    num_chunks = len(original_order_chunks_no_window)
    smeared_chunks = [torch.zeros_like(original_order_chunks_no_window[0]) 
                     for _ in range(num_chunks)]
    for position_idx in tqdm(range(num_chunks), total=num_chunks):
        source_chunk_idx = sort_order[position_idx]
        
        if dynamic_smear_width_embeds is not None:
            source_chunk_embed = all_features[source_chunk_idx]
            logits = source_chunk_embed @ dynamic_smear_width_embeds[0].T
            #print(logits)
            #print(logits.softmax(dim=0))
            smear_width = torch.sum((logits / logits.sum()) 
                                              * torch.tensor(dynamic_smear_width_embeds[1]).to(device)).item()
            print(smear_width)
            smear_width = max(round(smear_width), 1)
            envelope = get_envelope(smear_width)
                    
        for smear_slot in range(-smear_width, smear_width+1):
            for spread_slot in range(-spread, spread+1):
                smeared_chunks[
                    wrap(position_idx+smear_slot+spread_slot, num_chunks)
                ] += (
                        original_order_chunks_no_window[
                            wrap(source_chunk_idx+spread_slot, num_chunks)]
                        * envelope[smear_slot+smear_width] 
                        * (1/((spread+1)*smear_width))
                )

    for smeared_chunk in smeared_chunks:
        yield smeared_chunk
        
    #for i 
smear_width = 1
spread = 4
smear_mode = 'in-out'

dynamic_smear_width_defs = [('dreamy, mysterious, sparse', 6), ('intense, emotional, screaming, wailing', 1)]
print(tokenizer('stuff', padding=True, return_tensors='pt').input_ids)

dynamic_smear_width_embeds = (
    torch.concat([get_text_features(s) for s, _ in dynamic_smear_width_defs]),
    [w for _, w in dynamic_smear_width_defs]
)

smeared_chunks = list(smear_2(stereo_chunks_no_window, sort_order=sort_order, smear_width=smear_width, spread=spread, smear_mode=smear_mode, 
                              dynamic_smear_width_embeds=dynamic_smear_width_embeds
                              ))
#print(Counter([len(c) for c in smeared_chunks]))
smeared = torch.cat(smeared_chunks, dim=1)
print(smeared.shape)
torchaudio.save(filename + f'-sorted-norm-bpm{bpm}-cb{chunk_beats}-ww{window_width}-smeared-sm_{smear_mode}-sw{smear_width}-spread_actual{spread}-dyn2.wav', smeared, sample_rate=sampling_rate)


tensor([[    0, 40110,     2]])
tensor([0., 1., 0.])


  0%|          | 0/148 [00:00<?, ?it/s]

3.025988817214966
2.9852519035339355
3.2653768062591553
2.4126291275024414
2.916337251663208
2.4362034797668457
2.5560402870178223
2.888916015625
3.590606689453125
2.3081581592559814
4.081085205078125
7.580249309539795
-10.736671447753906
4.112405776977539
3.9278945922851562
3.7751708030700684
5.042652130126953
4.152952671051025
4.365510940551758
3.928717613220215
4.462410926818848
4.296231269836426
2.9200239181518555
3.811128616333008
3.860072374343872
4.524397850036621
4.241306781768799
3.9809253215789795
3.981428623199463
3.6327860355377197
3.5670528411865234
3.513566493988037
3.828505754470825
4.05603551864624
3.862551689147949
4.121306896209717
4.595206260681152
4.924497127532959
5.14949369430542
4.538237571716309
5.245449066162109
4.736452102661133
4.827077388763428
4.4414873123168945
5.3326416015625
5.198671340942383
5.352368354797363
6.101564884185791
5.535776615142822
5.547762393951416
6.531044006347656
6.016571044921875
5.717024326324463
9.459341049194336
6.713302135467529
92

In [214]:
from typing import Literal


def smear(chunks, smear_width):
    envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
    #for i in range(smear_width+1, 2*smear_width+1):
    #    envelope[i] = 0
    print(envelope)
    for i in range(len(chunks)-smear_width*2-1):
        smear_sources = [
            chunks[i+chunk_index] * envelope[chunk_index]
            #(i+chunk_index, envelope[chunk_index])
            for chunk_index in range(0, min(len(chunks), smear_width*2+1))
        ]
        #print(smear_sources)
        #print(set([ss.shape for ss in smear_sources]))
        #print(torch.sum(torch.stack(smear_sources), dim=0))
        smeared_chunk = torch.sum(torch.stack(smear_sources), dim=0) / len(smear_sources)
        #print(smeared_chunk.shape)
        #break
        yield smeared_chunk
        
def smear_2(original_order_chunks_no_window, sort_order, smear_width, spread=4, 
            smear_mode=Literal['in', 'in-out'],
            dynamic_smear_width_embeds: tuple[torch.Tensor, list[int], list[int]]=None):
    
    def get_envelope(smear_width):
        envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
        if smear_mode == 'in':
            for i in range(smear_width+1, 2*smear_width+1):
                envelope[i] = 0
        return envelope
    envelope = get_envelope(smear_width)
    print(envelope)
    num_chunks = len(original_order_chunks_no_window)
    smeared_chunks = [torch.zeros_like(original_order_chunks_no_window[0]) 
                     for _ in range(num_chunks)]
    for position_idx in tqdm(range(num_chunks), total=num_chunks):
        source_chunk_idx = sort_order[position_idx]
        
        if dynamic_smear_width_embeds is not None:
            source_chunk_embed = all_features[source_chunk_idx]
            logits = source_chunk_embed @ dynamic_smear_width_embeds[0].T
            #print(logits)
            #print(logits.softmax(dim=0))
            logits_norm = logits / logits.sum()
            smooth_dynamic_smear = True
            if smooth_dynamic_smear:
                smear_width = torch.sum(
                    logits_norm 
                    * torch.tensor(dynamic_smear_width_embeds[1]).to(device)
                ).item()
                spread = torch.sum(
                    logits_norm 
                    * torch.tensor(dynamic_smear_width_embeds[2]).to(device)
                ).item()
                print(smear_width, spread)
                smear_width = max(round(smear_width), 1)
                spread = max(round(spread), 1)
            else:
                smear_width = dynamic_smear_width_embeds[1][int(logits_norm.argmax().item())]
                spread = dynamic_smear_width_embeds[2][int(logits_norm.argmax().item())]
                print(smear_width, spread)
            envelope = get_envelope(smear_width)
        #volume_scaling = 1/envelope.sum()
                    
        for smear_slot in range(-smear_width, smear_width+1):
            for spread_slot in range(-spread, spread+1):
                smeared_chunks[
                    wrap(position_idx+smear_slot+spread_slot, num_chunks)
                ] += (
                        original_order_chunks_no_window[
                            wrap(source_chunk_idx+spread_slot, num_chunks)]
                        * envelope[smear_slot+smear_width] 
                        * (1/((spread+1)*envelope.sum()))
                )

    for smeared_chunk in smeared_chunks:
        yield smeared_chunk
        
    #for i 
spread = 4
smear_width = 1
smear_mode = 'in-out'

dynamic_smear_width_defs = [('dreamy, mysterious, sparse', 6, 1), 
                            ('driving, rhythmic, instrumental, simple beat', 2, 2), 
                            ('intense, emotional, screaming, wailing', 1, 6)]
print(tokenizer('stuff', padding=True, return_tensors='pt').input_ids)

dynamic_smear_width_embeds = (
    torch.concat([get_text_features(t) for t, _, _ in dynamic_smear_width_defs]),
    [w for _, w, _ in dynamic_smear_width_defs],
    [s for _, _, s in dynamic_smear_width_defs]
)

smeared_chunks = list(smear_2(stereo_chunks_no_window, sort_order=sort_order, smear_width=smear_width, spread=spread, smear_mode=smear_mode, 
                              dynamic_smear_width_embeds=dynamic_smear_width_embeds
                              ))
#print(Counter([len(c) for c in smeared_chunks]))
smeared = torch.cat(smeared_chunks, dim=1)
print(smeared.shape)
torchaudio.save(filename + f'-sorted-norm-bpm{bpm}-cb{chunk_beats}-ww{window_width}-smeared-sm_{smear_mode}-sw{smear_width}-spread_actual{spread}-dyn3d.wav', smeared, sample_rate=sampling_rate)


tensor([[    0, 40110,     2]])
tensor([0., 1., 0.])


  0%|          | 0/148 [00:00<?, ?it/s]

2.4786715507507324 2.9209675788879395
2.489936351776123 3.0018739700317383
2.7422468662261963 3.017498254776001
2.235888957977295 3.479130268096924
2.5422704219818115 3.2330713272094727
2.246544361114502 3.4490702152252197
2.2521414756774902 3.1082358360290527
2.3773345947265625 2.8961308002471924
2.4339187145233154 2.384483814239502
2.0919060707092285 2.802823781967163
2.621413230895996 2.274388313293457
3.0837583541870117 1.498881459236145
2.3414859771728516 1.578080177307129
2.952702522277832 2.400308132171631
2.7558298110961914 2.420318365097046
2.7713584899902344 2.5322206020355225
3.122119903564453 1.9842698574066162
2.856539726257324 2.3369927406311035
3.017334222793579 2.272874355316162
2.782302141189575 2.4345202445983887
2.9405951499938965 2.2053489685058594
2.8800487518310547 2.2697250843048096
2.408787727355957 2.9241814613342285
2.615767478942871 2.404205799102783
2.7111291885375977 2.4358086585998535
3.0090394020080566 2.190105438232422
2.967474937438965 2.327494621276855

In [210]:
torch.tensor([1, 2, 3]).argmax()

tensor(2)

In [ ]:
from typing import Literal
import math

def smear(chunks, smear_width):
    envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
    #for i in range(smear_width+1, 2*smear_width+1):
    #    envelope[i] = 0
    print(envelope)
    for i in range(len(chunks)-smear_width*2-1):
        smear_sources = [
            chunks[i+chunk_index] * envelope[chunk_index]
            #(i+chunk_index, envelope[chunk_index])
            for chunk_index in range(0, min(len(chunks), smear_width*2+1))
        ]
        #print(smear_sources)
        #print(set([ss.shape for ss in smear_sources]))
        #print(torch.sum(torch.stack(smear_sources), dim=0))
        smeared_chunk = torch.sum(torch.stack(smear_sources), dim=0) / len(smear_sources)
        #print(smeared_chunk.shape)
        #break
        yield smeared_chunk
        
def smear_2(original_order_chunks_no_window, sort_order, smear_width, spread=4, 
            smear_mode=Literal['in', 'in-out'],
            dynamic_smear_width_embeds: tuple[torch.Tensor, list[int], list[int]]=None):
    
    def get_envelope(smear_width):
        envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
        if smear_mode == 'in':
            for i in range(smear_width+1, 2*smear_width+1):
                envelope[i] = 0
        return envelope
    envelope = get_envelope(smear_width)
    print(envelope)
    num_chunks = len(original_order_chunks_no_window)
    smeared_chunks = [torch.zeros_like(original_order_chunks_no_window[0]) 
                     for _ in range(num_chunks)]
    for position_idx in tqdm(range(num_chunks), total=num_chunks):
        source_chunk_idx = sort_order[position_idx]
        
        if dynamic_smear_width_embeds is not None:
            source_chunk_embed = all_features[source_chunk_idx]
            logits = source_chunk_embed @ dynamic_smear_width_embeds[0].T
            #print(logits)
            #print(logits.softmax(dim=0))
            logits_norm = logits / logits.sum()
            smear_width = torch.sum(
                logits_norm 
                * torch.tensor(dynamic_smear_width_embeds[1]).to(device)
            ).item()
            spread = torch.sum(
                logits_norm 
                * torch.tensor(dynamic_smear_width_embeds[2]).to(device)
            ).item()
            print(smear_width, spread)
            smear_width = max(round(smear_width), 1)
            spread = max(round(spread), 1)
            envelope = get_envelope(smear_width)
                    
        for smear_slot in range(-smear_width, smear_width+1):
            for spread_slot in range(-spread, spread+1):
                smeared_chunks[
                    wrap(position_idx+smear_slot+spread_slot, num_chunks)
                ] += (
                        original_order_chunks_no_window[
                            wrap(source_chunk_idx+spread_slot, num_chunks)]
                        * envelope[smear_slot+smear_width] 
                        * (1/((spread+1)*smear_width))
                )

    for smeared_chunk in smeared_chunks:
        yield smeared_chunk
        
    #for i 
smear_width = 1
spread = 4
smear_mode = 'in-out'

dynamic_smear_width_defs = [('dreamy, mysterious, sparse', 6, 4), 
                            ('driving, rhythmic, instrumental, simple beat', 2, 2), 
                            ('intense, emotional, screaming, wailing', 1, 6)]
print(tokenizer('stuff', padding=True, return_tensors='pt').input_ids)

dynamic_smear_width_embeds = (
    torch.concat([get_text_features(t) for t, _, _ in dynamic_smear_width_defs]),
    [w for _, w, _ in dynamic_smear_width_defs],
    [s for _, _, s in dynamic_smear_width_defs]
)

smeared_chunks = list(smear_2(stereo_chunks_no_window, sort_order=sort_order, smear_width=smear_width, spread=spread, smear_mode=smear_mode, 
                              dynamic_smear_width_embeds=dynamic_smear_width_embeds
                              ))
#print(Counter([len(c) for c in smeared_chunks]))
smeared = torch.cat(smeared_chunks, dim=1)
print(smeared.shape)
torchaudio.save(filename + f'-sorted-norm-bpm{bpm}-cb{chunk_beats}-ww{window_width}-smeared-sm_{smear_mode}-sw{smear_width}-spread_actual{spread}-dyn3b.wav', smeared, sample_rate=sampling_rate)
